<a href="https://colab.research.google.com/github/AcidCannon/CMPUT466-Mini-project/blob/master/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# mount google drive to save checkpoints
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
config = {}
config['training_size'] = 60000
config['test_size'] = 5000
config['validation_size'] = 5000
config['training_shuffle'] = True
config['test_shuffle'] = False
config['validation_shuffle'] = True
config['num_of_classes'] = 10
config['device'] = 'gpu'
config['algorithm'] = 'Convolutional Neural Network'
config['training_batch_size'] = 64
config['validation_batch_size'] = 64
config['test_batch_size'] = 1
config['learning_rate'] = 0.0001
config['weight_decay'] = 0.001
config['best_epoch'] = -1
config['best_accuracy'] = 0
config['number_of_epochs'] = 100
config['display_progress_per_epoch'] = 1

In [0]:
def load_data(config):
  CIFAR10_training_set = datasets.CIFAR10('data', train=True, download=True, transform=transforms.Compose([
                                          transforms.RandomCrop(32, padding=4),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                        ]))
  
  CIFAR10_test = datasets.CIFAR10('data', train=False, download=True, transform=transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                        ]))
  
  CIFAR10_training_set = torch.utils.data.Subset(CIFAR10_training_set, range(0, config['training_size']))
  CIFAR10_test_set = torch.utils.data.Subset(CIFAR10_test, range(0, config['test_size']))
  CIFAR10_validation_set = torch.utils.data.Subset(CIFAR10_test, range(config['test_size'], config['test_size'] + config['validation_size']))

  train_dataloader = torch.utils.data.DataLoader(CIFAR10_training_set, batch_size=config['training_batch_size'], shuffle=config['training_shuffle'])
  valid_dataloader = torch.utils.data.DataLoader(CIFAR10_validation_set, batch_size=config['validation_batch_size'], shuffle=config['validation_shuffle'])
  test_dataloader = torch.utils.data.DataLoader(CIFAR10_test_set, batch_size=config['test_batch_size'], shuffle=config['test_shuffle'])
  return training_dataloader, test_dataloader, validation_dataloader,

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    pass
  
  def forward(self, x):
    pass

In [0]:
def train(training_dataloader, test_dataloader, config, device):
  model = Net().to(device)
  
  loss_function = F.cross_entropy
  optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay = config['weight_decay'])

  for epoch in range(1, config['number_of_epochs']+1):
    # TRAIN
    model.train()
    for idx, (data, target) in enumerate(training_dataloader):
      data = data.to(device)
      target = target.to(device)
      # set gradient to be zero
      optimizer.zero_grad()
      # forward propagation
      output = model(data)
      # compute loss
      loss = loss_function(output, target)
      # backward propagation
      loss.backward()
      # weight updation
      optimizer.step()
    # VALIDATION
    if epoch % config['display_progress_per_epoch'] == 0:
      model.eval()
      validation_loss = 0
      correct = 0
      # no gradients needed since we are doing validation
      with torch.no_grad():
        for idx, (data, target) in enumerate(test_dataloader):
          data = data.to(device)
          target = target.to(device)
          # predict / forward propagation
          output = model(image)
          loss = loss_function(output, target)
          # find target label
          _, predicted = torch.max(output.data, 1)
          correct += (predicted == target).sum()
          validation_loss += loss.item()
      accuracy = 100*correct/len(test_dataloader.dataset)
      print("Epoch: {}.\tAvg.loss: {}.\tAccuracy: {}%.".format(epoch, validation_loss/len(test_dataloader.dataset), accuracy))
      # checkpoint saving strategy: best validation accuracy
      if accuracy > config['best_accuracy']:
        config['best_epoch'] = epoch
        config['best_accuracy'] = accuracy
        torch.save(model.to(torch.device('cpu')), '/content/gdrive/My Drive/checkpoints/model.pt.{}'.format(epoch))
        print("Checkpoint saved.\tAt epoch: {}.\tWith accuracy: {}.".format(epoch, accuracy))
  return model

In [0]:
def test(model, test_dataloader, device):
  correct = 0
  total = len(test_dataloader.dataset)
  model.eval()
  with torch.no_grad():
    for idx, (data, target) in enumerate(test_dataloader):
      data = data.to(device)
      target = target.to(device)
      output = model(image)
      _, predicted = torch.max(output.data, 1)
      correct += (predicted == target).sum().item()
  return 100.*correct/total, correct, total

In [0]:
def run(config):
  if config['device'] != 'cpu' and torch.cuda.is_available():
    config['device'] = torch.device('cuda')
    print('Using GPU: {}.'.format(torch.cuda.get_device_name(0)))
  else:
    config['device'] = torch.device('cpu')
    print('Using CPU.')

  # training_dataloader
  